# VPC 생성 실습
- VPC 관련 몇 가지 알아둘 점
    - 커스텀 VPC 생성 시 만들어지는 리소스 : 라우팅 테이블, 기본 NACL, 기본 보안그룹
        - 서브넷 생성 시 모두 기본 라우팅 테이블로 자동 연동
    - VPC에는 단 하나의 Internet Gateway만 생성 가능
        - Internet Gateway 생성 후 직접 VPC에 연동 필요
        - Internet Gateway는 자체적으로 고가용성/장애내구성을 확보
    - 보안 그룹은 VPC 단위
    - 서브넷은 가용영역 단위(1서브넷 : 1가용영역)
    - 가장 작은 서브넷 단위는 /28(11개 IP = 2^4-5)

## 실습 순서
- 직접 커스텀 VPC 생성 (10.0.0.0/16)
- 서브넷 3개 생성 : Public, Web, DB
- 각 서브넷에 각각 Routing Table 연동
- Public 서브넷에 인터넷 경로 구성 : Internet Gateway 
- 퍼블릭 서브넷에 인스턴스 프로비전
- 프라이빗 서브넷에 인스턴스 프로비전
    - Bastion Host 사용
    - 해당 인스턴스에 웹 서버 설치
        - NAT Gateway Setup
    - Curl로 조회 테스트
- DB Subnet에 DB 설정 후 퍼블릭 인스턴스에서 접근

## VPC 직접 구성하기
- 생성 방법은 두 가지
    - VPC만 생성
    - VPC와 다른 리소스 같이 생성 (쉽다!)
        - 서브넷, 라우팅테이블, 네트워크 연결 등이 쉽게 생성됨
    - 여기서는 VPC만 생성하고 이후 리소스 수동으로 프로비전

# 실습 시작!
- 리전은 도쿄
1. VPC 생성
    - 생성할 리소스 : VPC만 선택
    - CIDR블록 수동으로 입력 (10.0.0.0/16)
    - 테넌시 기본값
        - 테넌시란? 클라우드 리소스의 분리 및 공유 방식에 대한 개념. 독립적인 리소스를 사용할 것인지, 공유된 리소스를 사용할 것인지.
    - 기본적으로 생성되는 리소스 : 라우팅 테이블, 보안 그룹(구분을 위해 이름을 붙여주자), NACL(얘도 이름 붙여주자)
2. 서브넷 생성(총 3개)
    - 서브넷 CIDR은 10.0.0.0/24, 10.0.1.0/24, 10.0.2.0/24 각각 다른 가용영역.
3. 서브넷에 라우팅 테이블 연결
    - 기본 라우팅 테이블에 자동으로(명시적 연결 없이)연결
4. 인터넷 게이트웨이 생성
    - 작업, VPC에 연결
5. 라우팅 테이블 만들기
    - 디폴트 라우팅 테이블의 이름을 프라이빗 라우팅 테이블로 바꿔주고(알아볼 수 있게)
    - 퍼블릭 라우팅 테이블 만들자
    - 디폴트 라우팅 테이블에 연결되어 있던 퍼블릭 서브넷에 퍼블릭 라우팅 테이블을 연결해주자
        - 라우팅 편집 : 모든 트래픽 인터넷 게이트웨이로 연결
        - 라우팅 테이블 서브넷 연결 : 퍼블릭 서브넷과 명시적 연결
    - 프라비잇 라우팅 테이블에도 서브넷들을 명시적 연결
6. 퍼블릭 서브넷 설정 편집
    - 퍼블릭 IP 자동 할당 활성화
7. 인스턴스 생성(퍼블릭)
    - 키페어 생성
    - 네트워크 설정 편집
        - VPC, 서브넷 선택
        - 보안그룹 디폴트
8. 디폴트 보안그룹 수정
    - 모든 트래픽 허용으로 수정
9. 퍼블릭 인스턴스에 연결, 웹서버 설치
    - 인스턴스의 퍼블릭 아이피로 접속이 잘 되는지 확인
10. 프라이빗 인스턴스(웹) 생성
    - 키페어 그대로 사용
    - VPC, 서브넷 선택
    - 보안그룹 디폴트
11. 배스쳔 호스트 생성
    - 퍼블릭 서브넷으로 접속(이름도 배스쳔 호스트로 변경)
    - 배스쳔 호스트에 키파일 써주기
        - sudo -s
        - nano keyfile.pem
        - chmod 400 keyfile.pem
        - ssh -i "keyfile.pem" ec2-user@privateIP
    - 현재 상태에서 웹 접속 확인 (불가능)
        - sudo -s
        - dnf install httpd -y
12. NAT GW 생성
    - 서브넷은 퍼블릭
    - 탄력적 IP 할당
    - 프라이빗 라우팅 테이블 편집
        - 모든 트래픽을 NAT GW로 연결
    - curl google.com 으로 인터넷 연결 확인
    - service httpd start 하고, 배스쳔 호스트에서 확인
    - nano /var/www/html/index.html 에 hello world 써놓고
    - curl privateIP



### DB 서버 구축 데모(mysql로 접근해보자)
1. 인스턴스 생성
    - 기존 키페어 사용
    - VPC, Subnet 선택
    - 디폴트 SG
2. Cloushell로 접속
    - create vpc environment
    - vpc, 서브넷, 디폴트 SG 선택
3. CloudShell에 입력
    - sudo -s
    - nano keyfile.pem
    - chmod 400 keyfile.pem
    - ssh -i "keyfile.pem" ec2-user@privateIP
4. 서버 설치
    - sudo -s
    - sudo wget https://dev.mysql.com/get/mysql80-community-release-el9-1.noarch.rpm 
    - sudo dnf install mysql80-community-release-el9-1.noarch.rpm -y
    - sudo rpm --import https://repo.mysql.com/RPM-GPG-KEY-mysql-2023
    - sudo dnf install mysql-community-server -y
    - service mysqld start
5. 클라이언트 설치
    - sudo dnf install mysql-community-client -y
    - grep 'temporary password' /var/log/mysqld.log (임시 패스워드 확인)
    - mysql -u root -p (이후 임시 패스워드 입력)
    - ALTER USER 'root'@'localhost' IDENTIFIED BY 'Audghks@490';
    - CREATE USER 'testuser'@'%' IDENTIFIED BY 'Audghks@490';
6. 퍼블릭 인스턴스에서 접속
    - sudo -s
    - sudo wget https://dev.mysql.com/get/mysql80-community-release-el9-1.noarch.rpm
    - sudo dnf install mysql80-community-release-el9-1.noarch.rpm -y
    - sudo rpm --import https://repo.mysql.com/RPM-GPG-KEY-mysql-2023
    - sudo dnf install mysql-community-client -y
    - mysql -h db서버ip -u testuser -p
7. 리소스 정리
    - 클라우드쉘 환경 삭제
    - 인스턴스들 모두 삭제
    - NAT GW 삭제
    - 탄력적 IP 릴리스, 삭제
    - VPC 삭제
